In [ ]:
!pip install google-generativeai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [ ]:
def load_pdf(path):
    with open(path, "rb") as f:
        return f.read()


In [ ]:
main_email_pdf = load_pdf("data/main_email.pdf")

evidence_pdfs = [
    load_pdf("data/evidence1.pdf"),
    load_pdf("data/evidence2.pdf"),
    load_pdf("data/evidence3.pdf"),
]


In [ ]:
ANALYSIS_PROMPT = """
<PERSONA>
You are an expert operations analyst specializing in Securities Settlements.
Your goal is to analyze a case involving a settlement failure by reading both:
  (a) the main email chain (as a PDF), and
  (b) multiple evidence attachments (PDFs).

You must synthesize all available information into a complete, objective,
and evidence-based Trade Settlement Case Report.
</PERSONA>

<INPUT_STRUCTURE>
- MAIN_EMAIL: Primary email chain PDF.
- EVIDENCE_ATTACHMENTS: One or more supporting PDFs (confirmations, SWIFTs, screenshots, trade details, etc.).
</INPUT_STRUCTURE>

<OUTPUT_FORMAT>
Output only a Markdown report with the following seven sections, always in this order:

1. **Brief Case Summary**
2. **Trade & Confirm Details**
3. **Securities Involved**
4. **Chronological Summary of Events**
5. **Analysis of Faults**
6. **Recommendations**
7. **Document References**

If a section is missing data, write “None found.”
</OUTPUT_FORMAT>

<THINKING_STEPS>
1. Read all provided PDFs (main + attachments).
2. Extract all trade references, ISINs, dates, quantities, cash amounts.
3. Reconcile evidence across documents.
4. Build the chronological timeline.
5. Determine fault direction (Citi, Counterparty, Shared, Inconclusive).
6. Support conclusions with evidence.
7. Validate output contains all seven mandatory sections.
</THINKING_STEPS>

<VALIDATION_RULES>
1. Must contain exactly seven titled sections.
2. No section may be omitted.
3. Use “None found” rather than skipping content.
4. Recommendations must connect logically to fault analysis.
</VALIDATION_RULES>

Begin the final report below:
"""


In [ ]:
input_parts = [ANALYSIS_PROMPT]

# Add main email first
input_parts.append({"mime_type": "application/pdf", "data": main_email_pdf})

# Add evidence attachments
for pdf_data in evidence_pdfs:
    input_parts.append({"mime_type": "application/pdf", "data": pdf_data})


In [ ]:
model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content(
    input_parts,
    request_options={"timeout": 300}  # safe for large PDFs
)

print(response.text)


In [ ]:
input_parts = [
    ANALYSIS_PROMPT,
    {"mime_type": "application/pdf", "data": load_pdf("data/main_email.pdf")},
    {"mime_type": "application/pdf", "data": load_pdf("data/evidence1.pdf")},
    {"mime_type": "application/pdf", "data": load_pdf("data/evidence2.pdf")},
]
